In [1]:
data_type = '17'
normalized = 'done'
erased_word = '' #JSNCFAO
num_word = 8000
import pickle
import os
path = os.getcwd()+'/pickles/'
with open(path+data_type+'training.pickle', 'rb') as f:
    training = pickle.load(f)
with open(path+data_type+'test.pickle', 'rb') as f:
    test = pickle.load(f)

In [2]:
with open(path+data_type+'sel_words'+str(num_word)+'_'+normalized+erased_word+'.pickle', 'rb') as f:
    selected_words = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\tmdgp\\python\\stock2/pickles/17sel_words8000_done.pickle'

In [3]:
# training set의 단어들만 따로 모아 학습에 포함할 단어 선별
import nltk
token = [t for d in training for t in d[0]] #단어들만 따로 모음
text = nltk.Text(token)
selected_words = [f[0] for f in text.vocab().most_common(num_word+(1000*len(erased_word)))] #상위 6000 빈도수 단어

if erased_word != '':
    word_for_remove = set()
    for word in selected_words:
        if 'J' in erased_word:
            if '/Josa' in word:
                word_for_remove.add(word)
        if 'S' in erased_word:
            if '/Suffix' in word:
                word_for_remove.add(word)
        if 'N' in erased_word:
            if '/Number' in word:
                word_for_remove.add(word)
        if 'C' in erased_word:
            if '/Conjunction' in word:
                word_for_remove.add(word)
        if 'F' in erased_word:
            if '/Foreign' in word:
                word_for_remove.add(word)
        if 'A' in erased_word:
            if '/Adverb' in word:
                word_for_remove.add(word)
        if 'O' in erased_word:
            if word[1] == '/':
                word_for_remove.add(word)
    for word in word_for_remove:
        selected_words.remove(word)

    # 수작업을 통한 필요한 단어 추가
    #selected_words.append('없이/Adverb')
        
    # 수작업을 통한 불필요한 단어 제거
    #selected_words.remove('하다/Verb')
    #selected_words.remove('있다/Adjective')
    #selected_words.remove('되다/Verb')
    #selected_words.remove('돼다/Verb')
    #selected_words.remove('이다/Verb')
selected_words = selected_words[:num_word]

In [6]:
# 선정된 각 단어들의 기사별 빈도수를 input data의 x값으로 정리
def term_frequency(doc):
    return [doc.count(word) for word in selected_words]


train_x_tmp = [(term_frequency(a),d) for a,_,d in training]
print('train_x_tmp is done')
test_x_tmp = [(term_frequency(a),d) for a,_,d in test]
print('test_x_tmp is done')

if normalized == 'done':
    # x 리스트의 총 합이 1이 되도록 normalize 하는 과정
    train_sum = [sum(d[0]) for d in train_x_tmp]
    print('train_sum is done')
    test_sum = [sum(d[0]) for d in test_x_tmp]
    print('test_sum is done')
    
    for i in range(len(train_sum)):
        if train_sum[i] == 0:
            train_sum[i] = 1
    for i in range(len(test_sum)):
        if test_sum[i] == 0:
            test_sum[i] = 1
    
    
    train_x = []
    train2_x = []    
    for d in range(len(train_x_tmp)):
        li = [float(l)/train_sum[d] for l in train_x_tmp[d][0]]
        train_x.append(li)
        train2_x.append((li,train_x_tmp[d][1]))
    print('train_x is done')
    test_x = []
    test2_x = []
    for d in range(len(test_x_tmp)):
        li = [float(l)/test_sum[d] for l in test_x_tmp[d][0]]
        test_x.append(li)
        test2_x.append((li,test_x_tmp[d][1]))
    print('test_x is done')
elif normalized == 'not':
    train_x = [a for a,_ in train_x_tmp]
    train2_x = train_x_tmp
    test_x = [a for a,_ in test_x_tmp]
    test2_x = test_x_tmp

train_x_tmp is done
test_x_tmp is done
train_sum is done
test_sum is done
train_x is done
test_x is done


In [7]:
# input data의 y값 정리
def make_y_list(data):
    if data == 0:
        return [0,1,0]
    elif data == 1:
        return [0,0,1]
    elif data == -1:
        return [1,0,0]
train_y = [make_y_list(c) for _,c,_ in training]
test_y = [make_y_list(c) for _,c,_ in test]
train2_y = {}
test2_y = {}
for _,c,d in training:
    train2_y[d]= int(c)
for _,c,d in test:
    test2_y[d]= int(c)

In [8]:
# 생성된 training, test의 x,y값을 pickle의 형태로 저장
import pickle
import os
path = os.getcwd()+'/pickles/'
if erased_word != '':
    erased_word = '_'+erased_word
with open(path+data_type+'train_x_'+str(num_word)+'_'+normalized+erased_word+'.pickle', 'wb') as f:
    pickle.dump(train_x, f)
with open(path+data_type+'train_y_'+str(num_word)+'_'+normalized+erased_word+'.pickle', 'wb') as f:
    pickle.dump(train_y, f)
with open(path+data_type+'train2_x_'+str(num_word)+'_'+normalized+erased_word+'.pickle', 'wb') as f:
    pickle.dump(train2_x, f)
with open(path+data_type+'train2_y_'+str(num_word)+'_'+normalized+erased_word+'.pickle', 'wb') as f:
    pickle.dump(train2_y, f)
with open(path+data_type+'test_x_'+str(num_word)+'_'+normalized+erased_word+'.pickle', 'wb') as f:
    pickle.dump(test_x, f)
with open(path+data_type+'test_y_'+str(num_word)+'_'+normalized+erased_word+'.pickle', 'wb') as f:
    pickle.dump(test_y, f)
with open(path+data_type+'test2_x_'+str(num_word)+'_'+normalized+erased_word+'.pickle', 'wb') as f:
    pickle.dump(test2_x, f)
with open(path+data_type+'test2_y_'+str(num_word)+'_'+normalized+erased_word+'.pickle', 'wb') as f:
    pickle.dump(test2_y, f)

In [9]:
with open(path+data_type+'sel_words'+str(num_word)+'_'+normalized+erased_word+'.pickle', 'wb') as f:
    pickle.dump(selected_words, f)